# Experiment with randomly selected parametered multiple runs for all strategies

In [7]:
symbols = ['BTC', 'ETH', 'BNB', 'XRP']

In [1]:
import random

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/BTC_6h.csv")
df.head()

,open_time,open,high,low,close,close_time,number_of_trades
0,2017-08-17 00:00:00,4261.48,4328.69,4261.32,4315.32,2017-08-17 05:59:59.999,273
1,2017-08-17 06:00:00,4330.29,4485.39,4287.41,4427.30,2017-08-17 11:59:59.999,309
2,2017-08-17 12:00:00,4436.06,4485.39,4235.45,4289.24,2017-08-17 17:59:59.999,1448
3,2017-08-17 18:00:00,4289.24,4369.69,4200.74,4285.08,2017-08-17 23:59:59.999,1397
4,2017-08-18 00:00:00,4285.08,4340.62,4134.61,4313.56,2017-08-18 05:59:59.999,1670


# Random all in buyer/seller

In [3]:
bank = 1000
wallet = 0

action = "buy"

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    if random.random() < 0.5:
        if action == "buy" and bank > 0:
            wallet = bank / price_to_buy_or_sell
            bank = 0
            action = "sell"
        elif action == "sell" and wallet > 0:
            bank = wallet * price_to_buy_or_sell
            wallet = 0
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank + wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 976.6341904864917
Total value: 379.13769659988185
Total value: 473.11474877013836
Total value: 631.3816848159414
Total value: 1500.3166342802695
Total value: 1117.0602736858173
Total value: 1197.1414935998444
Total value: 1005.6708196302357
Total value: 908.154837035185
Total value: 1301.9936738847741
Total value: 1685.4131678610565
last day: 1925.2603663332673


# Random portion buyer/seller

In [4]:
bank = 1000
wallet = 0

action = "buy"

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    if random.random() < 0.5:
        portion_to_use = random.random()

        if action == "buy" and bank > 0:
            wallet = wallet + (bank*portion_to_use) / price_to_buy_or_sell
            bank = bank - bank*portion_to_use
            action = "sell"
        elif action == "sell" and wallet > 0:
            bank = bank + wallet * portion_to_use * price_to_buy_or_sell
            wallet = wallet - wallet*portion_to_use
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 1588.7771004699694
Total value: 989.8126556976147
Total value: 1612.9026551222732
Total value: 1718.450271864297
Total value: 3754.4611430909854
Total value: 4874.329731765647
Total value: 3960.5716152716013
Total value: 3758.479495663274
Total value: 4005.5476529030266
Total value: 6704.114971410295
Total value: 7731.089762452435
last day: 8061.147764232495


# holder buyer/seller

In [5]:
bank = 1000
wallet = 0
set_buying_period = 24
prev_price_to_buy_or_sell = np.inf
buying_period = set_buying_period
action = "buy"

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    # if it's time to buy, and we have money
    if action == "buy" and bank > 0:
        wallet = bank / row_data["low"]
        bank = 0
        action = "sell"
        prev_price_to_buy_or_sell = price_to_buy_or_sell
        buying_period = set_buying_period

    # either we bought and now waiting for hold times to pass
    # or we are waiting for
    if buying_period != 0:
        buying_period -= 1
        continue

    # if it's time to sell, and we have coin, and we can profit from selling
    if action == "sell" and wallet > 0 and price_to_buy_or_sell > prev_price_to_buy_or_sell:
        bank = wallet * row_data["high"]
        wallet = 0
        action = "buy"
        buying_period = set_buying_period
    
    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 3529.7274676095
Total value: 1394.4998528342946
Total value: 3898.052319860154
Total value: 3601.8682477756206
Total value: 21035.496710833668
Total value: 32695.921348260676
Total value: 24264.848361384462
Total value: 18854.255094670316
Total value: 23292.870789883367
Total value: 64460.723081294476
Total value: 107454.65384366391
last day: 119775.25374975525


# Martingale betting

In [6]:
import numpy as np

bank = 1000  # Starting money
wallet = 0   # Amount of coin
set_holding_period = 10
base_bet = 10  # Initial bet
current_bet = base_bet
prev_price_to_buy = np.inf
holding_period = 0

action = "buy"

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    # Handle buy logic
    if action == "buy" and bank >= current_bet:
        coins_bought = current_bet / price_to_buy_or_sell
        wallet += coins_bought
        bank -= current_bet
        prev_price_to_buy = price_to_buy_or_sell
        holding_period = set_holding_period
        action = "sell"
    
    # Wait for the holding period to pass
    elif action == "sell":
        holding_period -= 1
        if holding_period > 0:
            continue
        else:
            # Holding period is over; decide whether to sell
            coin_value_now = wallet * price_to_buy_or_sell
            cost_basis = coins_bought * prev_price_to_buy

            profit = coin_value_now - cost_basis

            if profit >= 0:
                pass  # Reset to base bet
            else:
                # Double the previous bet or bet enough to cover the loss
                current_bet = current_bet * 2
                if current_bet > bank:
                    bank = bank + wallet * price_to_buy_or_sell
                    wallet = 0
                    if current_bet > bank:
                        current_bet = base_bet
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 1590.930290086436
Total value: 634.5827977081411
Total value: 1755.8993764700456
Total value: 1623.2414374941782
Total value: 6227.278863469632
Total value: 7226.562565738126
Total value: 4955.803309391671
Total value: 3852.98454515521
Total value: 4757.689156020951
Total value: 10748.895636937128
Total value: 14310.093870465354
last day: 15949.722577974719


# Dollar cost averaging

In [21]:
bank = 1000
wallet = 0
period = 1000
periodic_investment_budget = 1000 / (len(df) / period)

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    if i % period == 0 and bank >= periodic_investment_budget:
        # Invest the fixed amount
        quantity_bought = periodic_investment_budget / price_to_buy_or_sell
        wallet += quantity_bought
        bank -= periodic_investment_budget
    
    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 1106.989828835646
Total value: 934.8789336353772
Total value: 1296.0348253712634
Total value: 1246.563675378751
Total value: 3216.833653668622
Total value: 3658.7376136237663
Total value: 2626.6308146874712
Total value: 2105.5846387856645
Total value: 2553.9245417821608
Total value: 5634.967943122918
Total value: 7495.788235219159
last day: 8352.537612917842


## SMA crossover

In [9]:
bank = 1000
wallet = 0
action = "buy"  # Initial action can be to buy if there's bank

short_window = 10
long_window = 30

# Calculate Simple Moving Averages
df['SMA_short'] =((df["open"] + df["close"] + df["low"] + df["high"]) / 4).rolling(window=short_window, min_periods=1).mean()
df['SMA_long'] = ((df["open"] + df["close"] + df["low"] + df["high"]) / 4).rolling(window=long_window, min_periods=1).mean()

for i in range(len(df)):
    row_data = df.iloc[i]
    price_to_buy_or_sell = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4

    # Check for crossover signals (avoiding the first few periods where SMAs are still calculating)
    if i > long_window:
        previous_row = df.iloc[i-1]
        short_above_long = previous_row['SMA_short'] < previous_row['SMA_long'] and row_data['SMA_short'] > row_data['SMA_long']
        short_below_long = previous_row['SMA_short'] > previous_row['SMA_long'] and row_data['SMA_short'] < row_data['SMA_long']

        if short_above_long and action == "buy" and bank > 0:
            # Buy with all available bank
            wallet = bank / price_to_buy_or_sell
            bank = 0
            action = "sell"

        elif short_below_long and action == "sell" and wallet > 0:
            # Sell all available wallet
            bank = wallet * price_to_buy_or_sell
            wallet = 0
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 2177.2180878903446
Total value: 1563.444564669182
Total value: 3615.4011714832723
Total value: 3558.1268154996355
Total value: 8728.868800347946
Total value: 8533.817381564153
Total value: 8276.775823994705
Total value: 7809.630308891603
Total value: 8128.153728418645
Total value: 18110.860613154673
Total value: 19299.968842766597
last day: 20200.549178107693


## Volatility

In [10]:
bank = 1000
wallet = 0
action = "buy"  # Initial action

# Calculate ATR
high_low = df['high'] - df['low']
high_close_prev = abs(df['high'] - df['close'].shift(1))
low_close_prev = abs(df['low'] - df['close'].shift(1))
tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
df['ATR'] = tr.rolling(window=16, min_periods=1).mean()

for i in range(len(df)):
    row_data = df.iloc[i]
    price = (row_data["open"] + row_data["close"] + row_data["low"] + row_data["high"]) / 4
    atr = row_data["ATR"]

    # Define dynamic volatility thresholds based on the mean ATR
    mean_atr = df['ATR'].iloc[:i].mean() if i > 0 else atr  # Use historical mean

    if mean_atr > 0:
        low_volatility_threshold = mean_atr * 1.25
        high_volatility_threshold = mean_atr * 2.5
    else:
        low_volatility_threshold = 0
        high_volatility_threshold = float('inf')

    # Buy logic: After a period of low volatility, expecting a breakout
    if atr < low_volatility_threshold and action == "buy" and bank > 0:
        # Buy with all available bank
        wallet = bank / price
        bank = 0
        action = "sell"

    # Sell logic: After a period of high volatility, expecting consolidation
    elif atr > high_volatility_threshold and action == "sell" and wallet > 0:
        # Sell all available wallet
        bank = wallet * price
        wallet = 0
        action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 19472.75461155805
Total value: 19472.75461155805
Total value: 23564.353381344834
Total value: 18943.366163957286
Total value: 4275.9792574807
Total value: 4275.9792574807
Total value: 8699.609677397013
Total value: 8969.180389011777
Total value: 8969.180389011777
Total value: 8694.21489599097
Total value: 5856.942973097878
last day: 6339.397128457123


# Trend following

In [11]:
bank = 1000
wallet = 0
action = "buy"  # Initial action

for i in range(len(df)):
    price = df['close'].iloc[i]

    # Check for buy signal
    if action == "buy" and bank > 0 and i >= 16:
        previous_price = df['close'].iloc[i - 16]
        percentage_change = (price - previous_price) / previous_price
        if percentage_change >= 0.05:
            # Buy with all available bank
            wallet = bank / price
            bank = 0
            action = "sell"

    # Check for sell signal
    elif action == "sell" and wallet > 0 and i >= 5:
        previous_price = df['close'].iloc[i - 5]
        percentage_change = (price - previous_price) / previous_price
        if percentage_change <= -0.05:
            # Sell all available wallet
            bank = wallet * price
            wallet = 0
            action = "buy"

    if i%1000 == 0 and i>0:
        print("Total value:", bank + wallet * price_to_buy_or_sell)

bank = bank +  wallet * price_to_buy_or_sell

print("last day:", bank)

Total value: 25227.219178628326
Total value: 1859.4048751134128
Total value: 33876.411364883876
Total value: 31807.05470737833
Total value: 21857.567147929924
Total value: 7125.101917530751
Total value: 6283.895963023996
Total value: 25873.87271055447
Total value: 19120.849956223457
Total value: 15712.081594665742
Total value: 14144.277561280182
last day: 12738.118559957422


## Toast Bread

A. 1- find the best profiting [lowest-highest] slice range
A. 2- set call -> buy at lowest, sell at highest
A. 3- remove this range from data, collect remainings ranges as a list of ranges
A. 4- continue doing this 1-2-3 steps at each range till there is no [lowest-highest] range left
A. 5- for all remaining slices do one of these:
A. 5.1- buy at open if you can sell at close or highest with profit
A. 5.2- buy at lowest if you can sell at close

When new data comes:
B. 1- you dont have any ongoing plan, you are all done with your [highest lowest] slice range
B. 2- then you do all this planning stuff again with the new data

or

C. 1- you have some ongoing plan, with your [highest lowest] slice range
C. 2- discard all plans other than the ongoin one
C. 3- check the state of your ongoing plan
C. 4.1- if you have a better selling point at a highest further on the timeline, change your current ongoing plan's selling point to that point, and discard any slice that comes before that. do the steps A-4-5 for the remaining ones
C. 4.2- if you don't have a better selling point at further on the timeline and your highest is still valid, continue with your plan and do the A-4-5 with remaining ones
C. 4.3, if you don't have a better selling point at further on the timeline and your highest is now invalid, find the highest selling point that you can find, set your ongoing plan's selling point to that point to get away with lowest money loss. and do the A-4-5 with remaining ones

set all buy and sell actions with an error margin